<a href="https://colab.research.google.com/github/ribesstefano/ml4justice-feature-predictor/blob/main/notebooks/ml4justice_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML for Justice

## Setup and Imports

In [1]:
!pip install -U datasets evaluate -qqq
!pip install -U accelerate -qqq
!pip install -U transformers -qqq
# !pip install -U sentence-transformers -qqq
# !pip install umap-learn -qqq
# !pip install plotly==5.18.0 -qqq
# !pip install -U optuna -qqq

In [2]:
!pip install bitsandbytes -qqq
!pip install peft -qqq
# !pip install git+https://github.com/huggingface/peft.git -qqq

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

dataset_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/processed/dataset-sentenze-omicidio-stradale.csv", index_col=0, encoding='utf-8')
dataset_df

,corte,numero sentenza,localita commerciale o luogo di lavoro,vittima eta avanzata,vittima eta giovane,lavori stradali di manutenzione,guida senza patente valida imputato,guida senza patente valida vittima,piu imputati,strada a doppio senso di marcia,...,andamento retromarcia imputato,fuga imputato,uso telefono alla guida,vittima perde controllo,imputato perde controllo,condanna,filename,text,raw text,facts
casi,,,,,,,,,,,,,,,,,,,,,
1,Cass. Penale,20253,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,20253 Cass. penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 03/05/2...,"1. La Corte d'Appello di Ancona, con sentenza ..."
2,Cass. penale,18284,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,18284 cass. pen.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 20/04/2...,"1. Con la sentenza indicata in epigrafe, la Co..."
3,Cass. penale,15832,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,15832 cass penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 05/04/2...,"1. La Corte d'appello di Messina, in parziale ..."
4,Cass. penale,17209,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,17209 Cass. penale.pdf,--- Incipit ::: Cassazione penale sez. IV - 0...,SENTENZA \nCassazione penale sez. IV - 04/04/2...,"1. La Corte d'appello di Lecce, Sezione distac..."
5,Corte Appello Ancona,2112,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,2112 corte appello Ancona.pdf,"--- Incipit ::: Corte appello - Ancona, 23/01...","SENTENZA \nCorte appello - Ancona, 23/01/2023,...","Incipit Corte appello - Ancona, 23/01/2023, n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,cass. Penale,19944,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,Cass. penale 19944.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 28/02/2...,1. L.G. ricorre avverso la sentenza della Cort...
78,cass. Penale,42017,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,Cass. penale 42017.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 29/09/2...,1. La Corte d'appello di Bologna con la pronun...
79,cass. Penale,44364,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,Cass. penale 44364.pdf,--- Incipit ::: Cassazione penale sez. IV - 2...,SENTENZA \nCassazione penale sez. IV - 26/10/2...,1. Con sentenza del 23 novembre 2022 la Corte ...


In [5]:
label_columns = [col for col in dataset_df.columns if col not in ['numero sentenza', 'filename', 'text', 'raw text', 'corte', 'target', 'casi', 'condanna', 'colpa', 'labels', 'score', 'summary', 'facts',]]
label_columns.sort()
for l in label_columns:
    print(l)

andamento retromarcia imputato
attraversamento corretto della vittima
attraversamento vittima con semaforo rosso
attraversamento vittima senza strisce o sbucata improvvisamente
centro abitato
colpo di sonno
eccesso verso imputato
eccesso verso vittima
fuga imputato
giorno
giovane eta imputato
guida brusca o manovra brusca della vittima
guida brusca o manovra brusca imputato
guida senza patente valida imputato
guida senza patente valida vittima
imputato perde controllo
imputato ubriaco o drogato
invasione corsia opposta imputato
lavori stradali di manutenzione
localita commerciale o luogo di lavoro
maltempo
mancanza cintura di sicurezza o casco vittima
neopat imputato
notte
ostacolo evitabile dall imputato
ostacolo prevedibile dall imputato
ostacolo prevedibile dalla vittima
piu imputati
poca illuminazione o scarsa visibilita
presenza rimorchio imputato
prossimita di un incrocio o curva
segnaletica non visibile o assente
strada a doppio senso di marcia
strada rovinata o bagnata
uso tele

Following [this guide](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb) (and [this one](https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb) sometimes...).

In [6]:
labels = label_columns # [label for label in test_dataset.features.keys() if label not in ['text']]
num_labels = len(labels)
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}
labels

['andamento retromarcia imputato',
 'attraversamento corretto della vittima',
 'attraversamento vittima con semaforo rosso',
 'attraversamento vittima senza strisce o sbucata improvvisamente',
 'centro abitato',
 'colpo di sonno',
 'eccesso verso imputato',
 'eccesso verso vittima',
 'fuga imputato',
 'giorno',
 'giovane eta imputato',
 'guida brusca o manovra brusca della vittima',
 'guida brusca o manovra brusca imputato',
 'guida senza patente valida imputato',
 'guida senza patente valida vittima',
 'imputato perde controllo',
 'imputato ubriaco o drogato',
 'invasione corsia opposta imputato',
 'lavori stradali di manutenzione',
 'localita commerciale o luogo di lavoro',
 'maltempo',
 'mancanza cintura di sicurezza o casco vittima',
 'neopat imputato',
 'notte',
 'ostacolo evitabile dall imputato',
 'ostacolo prevedibile dall imputato',
 'ostacolo prevedibile dalla vittima',
 'piu imputati',
 'poca illuminazione o scarsa visibilita',
 'presenza rimorchio imputato',
 'prossimita di

## Text Utilities

In [7]:
sentence_div = '|||'
section_div = '---'
section_name_div = ':::'

def clean_text(text):
    text = text.replace(sentence_div, "").replace(section_div, "")
    return text.replace(section_name_div, "").strip()

def clean_section_text(text):
    return text.replace(sentence_div, "").strip()

def get_senteces_from_text(text, clean=True):
    sentences = text.split(sentence_div)
    sentences = [s.split(section_name_div)[-1].strip() for s in sentences]
    if clean:
        sentences = [clean_section_text(s) for s in sentences]
    return sentences

def get_sections_from_text(text):
    sections = text.split(section_div)
    return {s.split(section_name_div)[0].strip(): s.split(section_name_div)[-1].strip() for s in sections}

for i, sample in dataset_df.iterrows():
    text = sample['text']
    # sentences = get_senteces_from_text(text)
    # for s in sentences:
    #     print(s)
    sections = get_sections_from_text(text)
    for k, v in sections.items():
        print(k, clean_section_text(v))
    break

 
Incipit Cassazione penale sez. IV - 03/05/2023, n. 20253 Intestazione LA CORTE SUPREMA DI CASSAZIONE  SEZIONE QUARTA PENALE  Composta dagli Ill.mi Sigg.ri Magistrati: Dott. PICCIALLI Patrizia - Presidente - Dott. PEZZELLA Vincenzo - Consigliere - Dott. D'ANDREA Alessadro - Consigliere - Dott. MICCICHE'Loredana - rel. Consigliere - Dott. NOCERA Andrea - Consigliere - ha pronunciato la seguente:
SENTENZA sul ricorso proposto da:  H.M. nato il (Omissis); avverso la sentenza del 12/04/2022 della CORTE APPELLO di ANCONA visti gli atti, il provvedimento impugnato e il ricorso; udita la relazione svolta dal Consigliere LOREDANA MICCICHE'; lette le conclusioni Pubblico Ministero, in persona del Sostituto Procuratore ASSUNTA COCOMELLO; lette le conclusioni dell'avv. Emilio Perfetti per la parte civile Associazione Italiana familiari e vittime della strada.
RITENUTO IN FATTO 1. La Corte d'Appello di Ancona, con sentenza in data 12 aprile 2022, confermava la sentenza del GUP presso il Tribunale

In [8]:
import random

random.seed(42)

def chunk_section(text, chunk_size=1024):
    """ Group the sections sentence into chunks of size chunk_size """
    # Split the text into sentences
    sentences = get_senteces_from_text(text)
    # Group the sentences into chunks of size chunk_size
    chunks = []
    curr_chunk = ""
    for sentence in sentences:
        if len(curr_chunk) + len(sentence) < chunk_size:
            curr_chunk += sentence + " "
        elif len(sentence) > chunk_size:
            if curr_chunk != "":
                # print(len(curr_chunk))
                chunks.append(curr_chunk)
            # Keep appending chunks of size chunk_size to the list, if the sentence is longer multiple times than chunk_size
            for i in range(0, len(sentence), chunk_size):
                # print(len(sentence[i:i+chunk_size]))
                chunks.append(sentence[i:i+chunk_size])
            curr_chunk = ""
        else:
            # print(len(curr_chunk))
            chunks.append(curr_chunk)
            curr_chunk = sentence + " "
    # print(len(curr_chunk))
    chunks.append(curr_chunk)
    return chunks

def get_facts(text):
    facts = get_sections_from_text(text)
    if 'Svolgimento del processo' in facts and 'Motivi della decisione' in facts:
        # text = facts['Svolgimento del processo'] + " " + facts['Motivi della decisione']
        text = facts['Svolgimento del processo']
    elif "RITENUTO IN FATTO" in facts:
        text = facts['RITENUTO IN FATTO']
    return text

def get_random_chunk(text, chunk_size=1024):
    """ Get a random chunk of size chunk_size from the text """
    text = clean_section_text(text)
    start = random.randint(0, max(0, len(text) - chunk_size - 1))
    # max_chunks = len(text) // chunk_size
    # start = random.randint(0, max_chunks - 1) * chunk_size
    end = start + min(chunk_size, len(text))
    return text[start:end]

for i, sample in dataset_df.iterrows():
    text = sample['text']
    chunk = get_random_chunk(get_facts(text))
    print(chunk)
    break

ione dell'art. 141 C.d.S. poiché alla guida della sua autovettura, nel percorrere la strada provinciale n. (Omissis), alla guida dell'autocarro (Omissis), aveva tenuto una velocità non adeguata e comunque pari a 80 Km/h non commisurata allo stato dei luoghi, in considerazione delle condizioni metereologiche e dell'ora serale, e, affrontando un tratto di strada curvilineo, aveva perso il controllo del veicolo che, capovolgendosi, rovinava contro un manufatto, così cagionando la morte per gravissimo trauma cranico del F.C., seduto al lato passeggero anteriore.  La Corte territoriale, nel respingere i motivi di gravame, riteneva esaustiva la motivazione della sentenza di prime cure in ordine alle modalità del sinistro, su cui si soffermava rispondendo alle osservazioni mosse dall'appellante; confermava la statuizione del primo giudice in ordine al mancato riconoscimento delle circostanze attenuanti generiche, la durata della sanzione amministrativa accessoria e l'ammontare delle spese liq

## Inference

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# model_name = "iproskurina/tda-itabert-ita-cola"
# model_name = "ARTeLab/mbart-summarization-mlsum"
# model_name = "papluca/xlm-roberta-base-language-detection" # Best one so far
model_name = "classla/xlm-roberta-base-multilingual-text-genre-classifier"

# tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
#     use_fast=True,
#     # max_length=512, # At least 512
# )

In [10]:
def load_model(model_name, get_pipeline=False):
    tokenizer = AutoTokenizer.from_pretrained(f"ribesstefano/{model_name}", use_fast=True)
    if get_pipeline:
        model = pipeline("text-classification", model=f"ribesstefano/{model_name}", top_k=False)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(
            f"ribesstefano/{model_name}",
            problem_type="multi_label_classification",
            num_labels=len(labels),
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True,
        )
    return tokenizer, model

version = "v0.1"
k = 0
model_id = f"RuleBert-{version}-k{k}"
tokenizer, model = load_model(model_id)

In [11]:
from datasets import load_from_disk

base_dir = "/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/datasets/"
ds_name = f"{base_dir}/dataset-omicidio-stradale-fold-{k}"
tokenized_dataset = load_from_disk(ds_name)
print(tokenized_dataset['train']['labels'].numpy().shape)
print(tokenized_dataset['val']['labels'].numpy().shape)
print(tokenized_dataset['test']['labels'].numpy().shape)

(8100, 44)
(150, 44)
(110, 44)


In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8100
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
})

For the evaluation, reproduce [this detailed analysis](https://colab.research.google.com/github/samlowe/go_emotions-dataset/blob/main/eval-roberta-base-go_emotions.ipynb#scrollTo=571CYWiY6pS7).

In [26]:
import torch
import numpy as np
from sklearn import metrics
from torch.utils.data import DataLoader

def get_probabilities(text, model, tokenizer=None, batch_size=32):
    if tokenizer:
        encoding = tokenizer(text, return_tensors="pt", padding="max_length")
        encoding = {k: v.to(model.device) for k, v in encoding.items()}
        outputs = model(**encoding)
        logits = outputs.logits
    else:
        encoding = text
        logits = []
        for batch in DataLoader(encoding, batch_size=batch_size):
            with torch.no_grad():
                batch = {k: v.to(model.device) for k, v in batch.items()}
                outputs = model(**batch)
                logits.append(outputs.logits)
        logits = torch.vstack(logits)
    # Apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.cpu()) # .squeeze().cpu())
    return probs

def get_predictions(probs, threshold=0.5):
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= threshold)] = 1
    return predictions

def get_predicted_labels(predictions, id2label=id2label):
    # Turn predicted ID's into actual label names
    labeled_predictions = []
    for pred in predictions:
        tmp = [id2label[idx] for idx, label in enumerate(pred) if label == 1.0]
        labeled_predictions.append(tmp)
    return labeled_predictions

def predict(text, model, tokenizer=None, threshold=0.5, batch_size=32, id2label=id2label):
    probs = get_probabilities(text, model, tokenizer=tokenizer, batch_size=batch_size)
    predictions = get_predictions(probs, threshold)
    labeled_predictions = get_predicted_labels(predictions, id2label=id2label)
    return {
        'probabilities': probs,
        'predictions': predictions,
        'labeled_predictions': labeled_predictions,
    }

# examples = [
#     "L'imputato ha contestato una colpa generica e la violazione dell'art. 141 C.d. S. perché alla guida della sua autovettura, nel percorrere la strada provinciale, aveva tenuto una velocità non adeguata.",
#     "L'imputato, sotto effetto di alcool, ha contestato una colpa generica e la violazione dell'art. 141 C.d. S. perché alla guida della sua autovettura, nel percorrere la strada provinciale fuori cittá, aveva tenuto una velocità non adeguata di giorno.",
# ]
# predictions, predicted_labels = get_predicted_labels(examples, model, tokenizer, threshold=0.4)
tmp = predict(tokenized_dataset['test'], model, threshold=0.4)
for k, v in tmp.items():
    print(k, len(v))

# for pred in predicted_labels:
#     for l in pred:
#         print('*', l)
#     print('-' * 80)

probabilities 110
predictions 110
labeled_predictions 110


Get all model predictions, _i.e._, probabilities:

In [34]:
from datasets import load_from_disk
from tqdm import tqdm

sets = ['val', 'test'] # ['train', 'val', 'test']
version = "v0.1"
base_dir = "/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/datasets/"

y_targets = {}
probabilities = {}

for k in tqdm(range(5), desc='Getting predicted probabilities'):
    # Load model
    model_id = f"RuleBert-{version}-k{k}"
    tokenizer, model = load_model(model_id)
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu').eval()
    # Load dataset
    ds_name = f"{base_dir}/dataset-omicidio-stradale-fold-{k}"
    tokenized_dataset = load_from_disk(ds_name)
    # Get model predicted probabilities
    probabilities[k] = {d: get_probabilities(tokenized_dataset[d], model, batch_size=64) for d in sets}
    y_targets_all[k] = {d: tokenized_dataset[d]['labels'].numpy() for d in sets}
    del model

Getting predicted probabilities: 100%|██████████| 5/5 [01:29<00:00, 18.00s/it]


In [28]:
def calc_label_metrics(label, y_targets, y_preds, threshold):
    return {
        "Label": label,
        "Accuracy": metrics.accuracy_score(y_targets, y_preds),
        "Precision": metrics.precision_score(y_targets, y_preds, zero_division=0),
        "Recall": metrics.recall_score(y_targets, y_preds, zero_division=0),
        "F1": metrics.f1_score(y_targets, y_preds, zero_division=0),
        "MCC": metrics.matthews_corrcoef(y_targets, y_preds),
        "Support": y_targets.sum(),
        "Threshold": threshold,
    }

In [ ]:
# y_targets_all = {}
# for k in range(5):
#     # Load dataset
#     ds_name = f"{base_dir}/dataset-omicidio-stradale-fold-{k}"
#     tokenized_dataset = load_from_disk(ds_name)
#     y_targets_all[k] = {d: tokenized_dataset[d]['labels'].numpy() for d in ['train', 'val', 'test']}

In [64]:
from tqdm import tqdm

per_label_results = []

for k in tqdm(range(5), desc='Getting results'):

    # y_targets_set = tokenized_dataset['val']['labels'].numpy()
    # y_preds_set, _ = get_predicted_labels(tokenized_dataset['val'], model, threshold=threshold)

    # print(f"Accuracy: {metrics.accuracy_score(y_targets_set, y_preds_set):.3f}")
    # print(f"Precision: {metrics.precision_score(y_targets_set, y_preds_set, average='macro', zero_division=0):.3f}")
    # print(f"Recall: {metrics.recall_score(y_targets_set, y_preds_set, average='macro', zero_division=0):.3f}")
    # print(f"F1: {metrics.f1_score(y_targets_set, y_preds_set, average='macro', zero_division=0):.3f}")

    for set_name in sets:
        y_targets_set = y_targets_all[k][set_name]

        for threshold in [0.0 + i * 0.1 for i in range(11)]:
            y_preds_set = get_predictions(probabilities[k][set_name], threshold)

            for label_index, label in enumerate(labels):
                y_targets, y_preds = y_targets_set[:, label_index], y_preds_set[:, label_index]
                tmp = calc_label_metrics(label, y_targets, y_preds, threshold)
                tmp['k'] = k
                tmp['Dataset'] = set_name
                per_label_results.append(tmp)

per_label_results = pd.DataFrame(per_label_results) # , index=labels)
# per_label_results = per_label_results[per_label_results['support'] > 0]
per_label_results.round(3) # .drop(columns=["Label"]).round(3))

Getting results: 100%|██████████| 5/5 [00:35<00:00,  7.01s/it]


,Label,Accuracy,Precision,Recall,F1,MCC,Support,Threshold,k,Dataset
0,andamento retromarcia imputato,0.133,0.133,1.0,0.235,0.0,20.0,0.0,0,val
1,attraversamento corretto della vittima,0.200,0.200,1.0,0.333,0.0,30.0,0.0,0,val
2,attraversamento vittima con semaforo rosso,0.067,0.067,1.0,0.125,0.0,10.0,0.0,0,val
3,attraversamento vittima senza strisce o sbucat...,0.067,0.067,1.0,0.125,0.0,10.0,0.0,0,val
4,centro abitato,0.267,0.267,1.0,0.421,0.0,40.0,0.0,0,val
...,...,...,...,...,...,...,...,...,...,...
4835,vittima eta avanzata,0.818,0.000,0.0,0.000,0.0,20.0,1.0,4,test
4836,vittima eta giovane,1.000,0.000,0.0,0.000,0.0,0.0,1.0,4,test
4837,vittima perde controllo,1.000,0.000,0.0,0.000,0.0,0.0,1.0,4,test
4838,vittima priva di giubbino e bretelle riflettenti,1.000,0.000,0.0,0.000,0.0,0.0,1.0,4,test


In [65]:
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1', 'MCC', 'Support']
aggregation = {m: ['mean', 'median', 'std'] for m in metrics_names}

filename = "/content/drive/MyDrive/Colab Notebooks/ml_for_justice/data/scores-per-label.csv"
tmp = per_label_results.groupby(['Dataset', 'Threshold', 'Label']).agg(aggregation)
tmp.to_csv(filename, encoding='utf-8', index=False)
tmp.round(3)

Accuracy  \
                                                                         mean   
Dataset Threshold Label                                                         
test    0.0       andamento retromarcia imputato                        0.000   
                  attraversamento corretto della vittima                0.000   
                  attraversamento vittima con semaforo rosso            0.000   
                  attraversamento vittima senza strisce o sbucata...    0.182   
                  centro abitato                                        0.091   
...                                                                       ...   
val     1.0       vittima eta avanzata                                  0.865   
                  vittima eta giovane                                   0.943   
                  vittima perde controllo                               0.958   
                  vittima priva di giubbino e bretelle riflettenti      0.957   
                  vittima ubriaca o drogata                             0.987   

                                                                             \
                                                                     median   
Dataset Threshold Label                                                       
test    0.0       andamento retromarcia imputato                      0.000   
                  attraversamento corretto della vittima              0.000   
                  attraversamento vittima con semaforo rosso          0.000   
                  attraversamento vittima senza strisce o sbucata...  0.182   
                  centro abitato                                      0.091   
...                                                                     ...   
val     1.0       vittima eta avanzata                                0.929   
                  vittima eta giovane                                 0.929   
                  vittima perde controllo                             0.933   
                  vittima priva di giubbino e bretelle riflettenti    0.929   
                  vittima ubriaca o drogata                           1.000   

                                                                             \
                                                                        std   
Dataset Threshold Label                                                       
test    0.0       andamento retromarcia imputato                      0.000   
                  attraversamento corretto della vittima              0.000   
                  attraversamento vittima con semaforo rosso          0.000   
                  attraversamento vittima senza strisce o sbucata...  0.000   
                  centro abitato                                      0.000   
...                                                                     ...   
val     1.0       vittima eta avanzata                                0.188   
                  vittima eta giovane                                 0.060   
                  vittima perde controllo                             0.038   
                  vittima priva di giubbino e bretelle riflettenti    0.039   
                  vittima ubriaca o drogata                           0.030   

                                                                     Precision  \
                                                                          mean   
Dataset Threshold Label                                                          
test    0.0       andamento retromarcia imputato                         0.000   
                  attraversamento corretto della vittima                 0.000   
                  attraversamento vittima con semaforo rosso             0.000   
                  attraversamento vittima senza strisce o sbucata...     0.182   
                  centro abitato                                         0.091   
...                                                             

In [66]:
tmp[tmp['Support']['mean'] > 0]

Accuracy  \
                                                                          mean   
Dataset Threshold Label                                                          
test    0.0       attraversamento vittima senza strisce o sbucata...  0.181818   
                  centro abitato                                      0.090909   
                  colpo di sonno                                      0.181818   
                  eccesso verso imputato                              0.636364   
                  giovane eta imputato                                0.181818   
...                                                                        ...   
val     1.0       vittima eta avanzata                                0.864762   
                  vittima eta giovane                                 0.942857   
                  vittima perde controllo                             0.958095   
                  vittima priva di giubbino e bretelle riflettenti    0.957143   
                  vittima ubriaca o drogata                           0.986667   

                                                                                \
                                                                        median   
Dataset Threshold Label                                                          
test    0.0       attraversamento vittima senza strisce o sbucata...  0.181818   
                  centro abitato                                      0.090909   
                  colpo di sonno                                      0.181818   
                  eccesso verso imputato                              0.636364   
                  giovane eta imputato                                0.181818   
...                                                                        ...   
val     1.0       vittima eta avanzata                                0.928571   
                  vittima eta giovane                                 0.928571   
                  vittima perde controllo                             0.933333   
                  vittima priva di giubbino e bretelle riflettenti    0.928571   
                  vittima ubriaca o drogata                           1.000000   

                                                                                \
                                                                           std   
Dataset Threshold Label                                                          
test    0.0       attraversamento vittima senza strisce o sbucata...  0.000000   
                  centro abitato                                      0.000000   
                  colpo di sonno                                      0.000000   
                  eccesso verso imputato                              0.000000   
                  giovane eta imputato                                0.000000   
...                                                                        ...   
val     1.0       vittima eta avanzata                                0.187736   
                  vittima eta giovane                                 0.059761   
                  vittima perde controllo                             0.038303   
                  vittima priva di giubbino e bretelle riflettenti    0.039123   
                  vittima ubriaca o drogata                           0.029814   

                                                                     Precision  \
                                                                          mean   
Dataset Threshold Label                                                          
test    0.0       attraversamento vittima senza strisce o sbucata...  0.181818   
                  centro abitato                                      0.090909   
                  colpo di sonno                                      0.181818   
                  eccesso verso imputato                              0.636364   
                  giovane eta imputato           

In [67]:
per_label_results[per_label_results['Support'] > 0].groupby(['Dataset', 'Threshold']).agg(aggregation).round(3)

Accuracy               Precision               Recall  \
                      mean median    std      mean median    std   mean   
Dataset Threshold                                                         
test    0.0          0.310  0.227  0.213     0.310  0.227  0.213  1.000   
        0.1          0.528  0.636  0.281     0.254  0.182  0.259  0.664   
        0.2          0.648  0.727  0.239     0.191  0.000  0.275  0.414   
        0.3          0.741  0.818  0.140     0.169  0.000  0.277  0.281   
        0.4          0.751  0.818  0.135     0.127  0.000  0.266  0.191   
        0.5          0.750  0.818  0.138     0.108  0.000  0.255  0.155   
        0.6          0.744  0.818  0.147     0.095  0.000  0.242  0.136   
        0.7          0.744  0.818  0.147     0.095  0.000  0.242  0.136   
        0.8          0.736  0.818  0.161     0.087  0.000  0.232  0.126   
        0.9          0.697  0.818  0.206     0.017  0.000  0.105  0.021   
        1.0          0.690  0.773  0.213     0.000  0.000  0.000  0.000   
val     0.0          0.248  0.143  0.238     0.248  0.143  0.238  1.000   
        0.1          0.604  0.707  0.285     0.239  0.147  0.275  0.542   
        0.2          0.734  0.847  0.223     0.234  0.036  0.306  0.351   
        0.3          0.786  0.857  0.171     0.254  0.000  0.343  0.247   
        0.4          0.805  0.857  0.145     0.232  0.000  0.354  0.157   
        0.5          0.816  0.857  0.131     0.188  0.000  0.355  0.119   
        0.6          0.820  0.857  0.127     0.145  0.000  0.332  0.104   
        0.7          0.821  0.857  0.128     0.113  0.000  0.296  0.101   
        0.8          0.816  0.857  0.130     0.086  0.000  0.258  0.095   
        0.9          0.765  0.857  0.214     0.061  0.000  0.218  0.022   
        1.0          0.752  0.857  0.238     0.000  0.000  0.000  0.000   

                                    F1                  MCC                \
                  median    std   mean median    std   mean median    std   
Dataset Threshold                                                           
test    0.0        1.000  0.000  0.437  0.368  0.229  0.000    0.0  0.000   
        0.1        1.000  0.470  0.341  0.308  0.313  0.000    0.0  0.089   
        0.2        0.000  0.492  0.245  0.000  0.334 -0.012    0.0  0.077   
        0.3        0.000  0.448  0.207  0.000  0.335  0.001    0.0  0.074   
        0.4        0.000  0.395  0.152  0.000  0.315 -0.001    0.0  0.056   
        0.5        0.000  0.363  0.127  0.000  0.298  0.008    0.0  0.056   
        0.6        0.000  0.345  0.112  0.000  0.283  0.000    0.0  0.000   
        0.7        0.000  0.345  0.112  0.000  0.283  0.000    0.0  0.000   
        0.8        0.000  0.332  0.103  0.000  0.271 -0.008    0.0  0.068   
        0.9        0.000  0.128  0.019  0.000  0.115 -0.007    0.0  0.055   
        1.0        0.000  0.000  0.000  0.000  0.000  0.000    0.0  0.000   
val     0.0        1.000  0.000  0.351  0.250  0.250  0.000    0.0  0.000   
        0.1        0.650  0.446  0.289  0.196  0.294  0.007    0.0  0.091   
        0.2        0.033  0.445  0.239  0.042  0.311  0.009    0.0  0.098   
        0.3        0.000  0.385  0.204  0.000  0.305  0.024    0.0  0.102   
        0.4        0.000  0.328  0.148  0.000  0.288  0.011    0.0  0.092   
        0.5        0.000  0.306  0.113  0.000  0.278  0.005    0.0  0.067   
        0.6        0.000  0.300  0.098  0.000  0.275  0.003    0.0  0.052   
        0.7        0.000  0.300  0.094  0.000  0.275  0.001    0.0  0.035   
        0.8        0.000  0.284  0.089  0.000  0.267  0.002    0.0  0.043   
        0.9        0.000  0.101  0.028  0.000  0.119 -0.002    0.0  0.032   
        1.0        0.000  0.000  0.000  0.000  0.000  0.000    0.0  0.000   

                     Support                 
                        mean median     std  
Dataset Threshold                            
test    0.0        34.091000   25.0  23.395  
        0.1        34.091000   2